In [6]:
import pandas as pd
import jieba
import jieba.posseg as pseg
import jieba.analyse

#import nltk
#from nltk.book import *
import jieba

#-*-coding:utf-8-*-

user_dict = './NLP with python/wpa_dict.txt'
stop_dict = './NLP with python/stop_word_dict.txt'
replace_file = './NLP with python/replace_file.txt'

In [50]:
#去除标点符号
def trim_puntuation(msg):
    import re 
    tmp_str = msg.decode("utf8")
    #print tmp_str
    puntuation = "[0-9\s+\.\!\/_,$%^*(+\"\'\\]+|[+——！，。？?、~@#￥%……&*（）]+".decode("utf8")
    #print puntuation   
    res_str = re.sub(puntuation, " ".decode("utf8"), tmp_str)
    return res_str

#过滤停用词  
import codecs
def trim_stop_word(word_list, stop_word_file):
    st = codecs.open(stop_word_file, 'rb',encoding='utf-8')      
    stopwords = [ line.strip() for line in st ]  
    return [ w for w in word_list if len(w.strip()) >0 and w not in stopwords ]

def fen_ci(msg):
    # print msg 
    # msg = str(msg).decode('utf8')
    text_pun = trim_puntuation(msg)
    word_list = trim_stop_word(jieba.lcut(text_pun), stop_dict)
    return ' '.join(word_list)

In [51]:
df = pd.read_csv('./NLP with python/wpa_msg_20171123.csv')
df.drop_duplicates(inplace=True)
#df.info()

# 只考虑C2B会话
c2b_df = df[df.is_b2c == 0]
c2b_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5232 entries, 0 to 7489
Data columns (total 14 columns):
stat_day           5232 non-null int64
kfuin              5232 non-null int64
qq                 5232 non-null int64
is_b2c             5232 non-null int64
is_delay           5232 non-null int64
is_not_ack         5232 non-null int64
msg_c2b            5232 non-null int64
msg_b2c            5232 non-null int64
recv_msg_num       5232 non-null int64
send_msg_num       5232 non-null int64
first_recv_date    5232 non-null object
first_send_date    3883 non-null object
is_spam            5232 non-null int64
msg                5232 non-null object
dtypes: int64(11), object(3)
memory usage: 613.1+ KB


In [57]:
jieba.load_userdict(user_dict)
df['words'] = map(fen_ci, map(str, df['msg']))

In [58]:
df['msg'][:10]

0                                               下   在 
1                                            多少一个企业QQ 
2     1   企业号可以找回吗   你好   e   11   这个   企业员工号被盗了怎么找...
3                                             营销QQ怎么卖 
4                                              zaima  
5                           我的QQ开了至尊保但被封了   QQ能换手机号码吗 
6                            在吗？   QQ空间被举报只有自己能进去看的了。 
7     我QQ  为啥加不上人？   我给100   300   500   600   1000...
8             在？   企业号可以开2000人群？   是的   把你的这么回复关了   0 
9                                       我想免费要一个5位号的qq 
Name: msg, dtype: object

In [59]:
df['words'][:10]

0                                                    下
1                                             一个 企业 QQ
2    企业 号 找回 你好 e 企业 员工 号 被盗 找回 手机 密码 改了 手机 绑定 还 找回...
3                                              营销 QQ 卖
4                                                zaima
5                              QQ 开 至尊 保但 封 QQ 能换 手机号码
6                                        QQ 空间 举报 进去 看
7                                     QQ 为啥 加不上 人 ko \
8                                       企业 号 开 人群 回复 关
9                                        想 免费 一个 位号 qq
Name: words, dtype: object

In [69]:
# 统计所有的字段的频率，给出top10
def word_freq_top(word_list, top_k=10):
    from collections import Counter
    wc = Counter(word_list)
    sort_list = sorted(wc.items(), key = lambda d:d[1], reverse=True)
    for w,c in sort_list[: top_k]:
        print "%s -> %d " %(w,c),
        
word_list = [w for words in df['words'][:100] for w in words.split()]
word_freq_top(word_list,20)

QQ -> 77  企业 -> 66  你好 -> 39  不 -> 36  好 -> 31  qq -> 30  申请 -> 29  没 -> 27  营销 -> 23  下 -> 22  想 -> 21  说 -> 19  网站 -> 15  一个 -> 15  都 -> 14  公司 -> 13  微信 -> 12  加 -> 12  谢谢 -> 12  价格 -> 11 


In [67]:
from collections import Counter
list1=['apple','egg','apple','banana','egg','apple']
word_freq_top(list1)

apple -> 3  egg -> 2  banana -> 1  企业 -> 7  QQ -> 7  手机 -> 4  工号 -> 4  一个 -> 4  号 -> 4  找回 -> 3  换 -> 3  开 -> 2  人 -> 2 


In [32]:
# 只有那些20171101-20171106这六天，那些客服没有回复的会话，都标注过

c2b_df['test_flag'] = 1
c2b_df.loc[(c2b_df.is_not_ack == 1) & (c2b_df.stat_day >= 20171101) 
           & (c2b_df.stat_day<=20171106) & (c2b_df.is_spam != 2),
           'test_flag' ] = 0 
train_df = c2b_df[c2b_df.test_flag == 0]
test_df = c2b_df[c2b_df.test_flag == 1]
print 'train_df.shape:', train_df.shape, 'test_df.shape:', test_df.shape

train_df.shape: (343, 15) test_df.shape: (4889, 15)


C:\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
train_df['is_spam'].value_counts()

1    261
0     82
Name: is_spam, dtype: int64

In [40]:
text = all_text[:1001]
print text

 下   在   没定单啊   不是   是申请的   网上申请的企业   qq   嗯嗯   多少钱呢   暂时2   个   多少一个企业QQ   你好   企业qq换手机在哪换   网址发一下   谢谢   ?   企业QQ换手机   怎么换   500人的企业号   多少钱一个   嗯   一个工号120?   那100个工号呢   有10个工号的吗   我人不是很多   只需10个工号   价格是多少   这个怎么购买呢   个人现在还不行吗   官网能发下吗   我找不到  nan  zaima   nan nan  我QQ  为啥加不上人？   我给100   300   500   600   1000   500000   56.636   6200000   ko?\   在？   企业号可以开2000人群？   是的   把你的这么回复关了   0  nan  你好   企业QQ30个多少钱一年   标准的   贵了   你这价格   有人买5000   扯蛋   百度我都查了   你应该是代理   你是代理还是   哦哦   代理也买6000   坑啊   能用一年就好了   不要了   你们自己用吧   那为什么人家买的比你


In [84]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?'
]

corpus = df['words'][:10]
vectorizer = CountVectorizer(ngram_range=(1,1), binary=True)
X = vectorizer.fit_transform(corpus)
X.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 1, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,

In [77]:
X.shape

(10, 63)

In [85]:
print '/'.join(vectorizer.get_feature_names())

ko/nan/qq/zaima/一个/不到/不行/为啥/人群/价格/企业/你好/加不上/发下/哪换/回复/多少钱/官网/定单/工号/很多/我人/手机/暂时/申请/网上/网址/谢谢/购买
